In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys, os
from pathlib import Path

In [ ]:
sys.path.insert(0, '../')

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.ERROR)

In [ ]:
import yaml

In [ ]:
BASE_PATH = Path("../../voicebot/config/")

# Parse input file

In [ ]:
with open(BASE_PATH / "strings.yml") as f:
    lines = f.readlines()
with open(BASE_PATH / "strings.yml") as f:
    yaml_data = yaml.safe_load(f)

In [ ]:
translations = {}
lines_it = iter(lines)
current_name = None
current_translation = None
while True:
    try:
        line = next(lines_it)
    except StopIteration:
        line = None

    if line is None or (len(line) > 0 and line.strip().endswith(":") and not line.startswith('  ')):
        if current_name is not None:
            current_translation = "\n".join(
                x[2:] if x.strip() else ""
                for x in current_translation.splitlines()
            )
            translations[current_name] = current_translation
        if line is not None:
            current_name = line.strip()[:-1]
            current_translation = ""
        else:
            break
    else:
        current_translation += line

# Manually choose which keys to translate

In [ ]:
keys_to_translate = [
    x for x in yaml_data
    # if len(yaml_data[x]) < 3
]

# Manually choose languages

In [ ]:
languages = DEFAULT_LANGUAGES
from iso639 import Lang
languages = [Lang("Chinese")]

# Do translation

In [ ]:
from llm_tools.ui_translator import (
    get_translation_yaml,
    DEFAULT_LANGUAGES,
)

In [ ]:
from llm_tools.llm_fallback import StreamingModelWithFallback
from llm_tools.llm_streaming import StreamingOpenAIChatModel
from langchain.chat_models import ChatOpenAI

In [ ]:
model = StreamingModelWithFallback(models=[StreamingOpenAIChatModel(
    ChatOpenAI(
        openai_api_key="<KEY>",
        model_name='gpt-4',
        streaming=True,
        temperature=0
    )
)])

In [ ]:
from IPython.display import clear_output

In [ ]:
new_translations = {}

In [ ]:
languages[0]

## Prints current translation to verify it's going OK

In [ ]:
for key in keys_to_translate[1:]:
    try:
        messages = get_translation_yaml(translations[key], languages=languages)
        print(messages[-1]["content"])
        async for text, token in model.stream_llm_reply(messages):
            print(token, end="")
        new_translations[key] = text
    finally:
        clear_output(wait=True)

# Dump result

In [ ]:
result = []
for name in translations:
    result.append(f"{name}:\n")
    if name in new_translations:
        translation = new_translations[name]
        lines = translation.splitlines()
        assert "```" in lines[0]
        assert "```" in lines[-1]
        lines = translations[name].splitlines() + lines[1:-1]
    else:
        translation = translations[name]
        lines = translation.splitlines()
    for line in lines:
        if line.strip():
            result.append(f"  {line}\n")
        else:
            result.append("\n")

In [ ]:
with open(BASE_PATH / 'strings_new.yml', "w") as f:
    f.write("".join(result))